In [ ]:
import os
import reservoirpy as rpy
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from sklearn.metrics import r2_score
from reservoirpy.nodes import Reservoir, Ridge
import logging
import gc
import pickle

# ログ設定
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# グローバル設定
rpy.verbosity(0)  # Verboseモードを無効
rpy.set_seed(42)  # 再現性のためのシード設定

In [ ]:
# パラメータ設定
warm_up = 1000  # 初期値の捨てる数
max_delay = 2000  # 遅延数
fit_window = 1000  # W_outのfitするwindow
n_timesteps = warm_up + max_delay + fit_window  # 全体のtimesteps

rc_connectivity_list = np.array([0.2, 0.4, 0.6, 0.8, 1.0])
units_list = np.array([100, 500, 1000, 2000])
spectral_radii = np.array([0.1, 0.25, 0.5, 0.75, 1.0, 1.25])
batch_size = 20  # バッチサイズを減らしてメモリ負荷軽減

# 結果保存先のディレクトリ
output_dir = "jupyter/src/memory_capacity/results/hyper_params_MC"
os.makedirs(output_dir, exist_ok=True)  # ディレクトリがなければ作成

In [ ]:
# データ生成
np.random.seed(42)
input_signal = np.random.randn(n_timesteps).reshape(-1, 1)  # ランダムな入力信号を生成
input_signal = (input_signal - np.mean(input_signal)) / np.std(input_signal)  # 標準化

In [ ]:
# ESN（Echo State Network）を作成
def create_standard_esn(spectral_radius: float, units: int, rc_connectivity: float) -> rpy.Network:
    logging.info(f"Creating ESN with spectral radius={spectral_radius}, units={units} and rc_connectivity={rc_connectivity}")
    try:
        reservoir = Reservoir(units=units, input_scaling=1, rc_connectivity=rc_connectivity, lr=1, sr=spectral_radius)
        readout = Ridge(output_dim=1, ridge=1e-5)
        return reservoir >> readout
    except Exception as e:
        logging.error(f"Failed to create ESN: {e}")
        return None

In [ ]:
def fit_with_delay(esn: rpy.Network, input_signal: np.ndarray, output_signal, warm_up, max_delay, fit_window):
    try:
        for start in range(0, max_delay, batch_size):
            end = min(start + batch_size, max_delay)
            r2_batch = Parallel(n_jobs=8, backend="loky")(delayed(compute_r2)(
                spectral_radius, units, rc_connectivity, X[d], Y[d]) for d in range(start, end))
            r2_scores.extend(r2_batch)
            gc.collect()  # 各バッチ後にメモリ解放
        memory_capacity = np.nansum(r2_scores)
        return {
            "Spectral Radius": spectral_radius,
            "Units": units,
            "RC Connectivity": rc_connectivity,
            "Memory Capacity": memory_capacity,
            "R2 Scores": r2_scores
        }
    except Exception as e:
        logging.error(f"Failed to compute memory capacity for sr={spectral_radius}, units={units}, rc_connectivity={rc_connectivity}: {e}")
        return None

# R^2スコアを計算
def compute_r2(spectral_radius: float, units: int, rc_connectivity: float, input_signal: np.ndarray) -> float:
    try:
        esn = create_standard_esn(spectral_radius, units, rc_connectivity)
        if esn is None:
            return np.nan
        
        pred = esn.run(input_signal)
        r2 = r2_score(output_signal, pred)
        del esn, pred
        gc.collect()
        return max(r2, 0)
    except Exception as e:
        logging.error(f"Failed to compute R2 score for sr={spectral_radius}, units={units}, rc_connectivity={rc_connectivity}: {e}")
        return np.nan

In [ ]:
# メモリ容量を計算
def compute_memory_capacity(spectral_radius, units, rc_connectivity, X, Y):
    logging.info(f"Calculating memory capacity for spectral radius={spectral_radius}, units={units} and rc_connectivity={rc_connectivity}")
    r2_scores = []
    try:
        for start in range(0, max_delay, batch_size):
            end = min(start + batch_size, max_delay)
            r2_batch = Parallel(n_jobs=8, backend="loky")(delayed(compute_r2)(
                spectral_radius, units, rc_connectivity, X[d], Y[d]) for d in range(start, end))
            r2_scores.extend(r2_batch)
            gc.collect()  # 各バッチ後にメモリ解放
        memory_capacity = np.nansum(r2_scores)
        return {
            "Spectral Radius": spectral_radius,
            "Units": units,
            "RC Connectivity": rc_connectivity,
            "Memory Capacity": memory_capacity,
            "R2 Scores": r2_scores
        }
    except Exception as e:
        logging.error(f"Failed to compute memory capacity for sr={spectral_radius}, units={units}, rc_connectivity={rc_connectivity}: {e}")
        return None

# 部分保存関数
def save_partial_results(result):
    try:
        spectral_radius = result["Spectral Radius"]
        units = result["Units"]
        rc_connectivity = result["RC Connectivity"]
        file_name = f"sr_{spectral_radius}_units_{units}_rc_{rc_connectivity}.pkl"
        path_name = os.path.join(output_dir, file_name)
        with open(path_name, "wb") as f:
            pickle.dump(result, f)
        logging.info("Partial result saved successfully.")
    except Exception as e:
        logging.error(f"Failed to save partial result: {e}")

# 全てのパラメータの組み合わせについてメモリ容量を計算
def compute_all_memory_capacities(spectral_radii, units_list, rc_connectivity_list, X, Y):
    for sr in spectral_radii:
        for units in units_list:
            for rc_connectivity in rc_connectivity_list:
                result = compute_memory_capacity(sr, units, rc_connectivity, X, Y)
                if result is not None:
                    save_partial_results(result)


In [ ]:
# 実行
all_results = compute_all_memory_capacities(spectral_radii, units_list, rc_connectivity_list, X, Y)
print("All memory capacity results have been calculated and saved.")

2024-11-13 11:41:56,713 - INFO - Calculating memory capacity for spectral radius=0.1, units=100 and rc_connectivity=0.2
2024-11-13 11:47:26,653 - INFO - Partial result saved successfully.
2024-11-13 11:47:26,655 - INFO - Calculating memory capacity for spectral radius=0.1, units=100 and rc_connectivity=0.4
2024-11-13 11:49:00,529 - ERROR - Failed to compute memory capacity for sr=0.1, units=100, rc_connectivity=0.4: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGBUS(-7)}
2024-11-13 11:49:00,531 - INFO - Calculating memory capacity for spectral radius=0.1, units=100 and rc_connectivity=0.6
2024-11-13 11:54:50,846 - INFO - Partial result saved successfully.
2024-11-13 11:54:50,848 - INFO - Calculating memory capacity for spectral radius=0.1, units=100 and rc_connectivity=0.8
2